In [1]:
import findspark
findspark.init("/usr/local/spark")

In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("nlp").getOrCreate()

In [28]:
data=spark.read.csv("/home/os/Desktop/SMSSpamCollection.csv",inferSchema=True,sep="\t")

In [33]:
data=data.withColumnRenamed('_c0','class').withColumnRenamed("_c1","text")

In [34]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [35]:
from pyspark.sql.functions import length 
data=data.withColumn("length",length(data['text'])) #length adında yeni sutun oluştur.

In [37]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [39]:
data.groupby("class").mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [40]:
from pyspark.ml.feature import (Tokenizer,StopWordsRemover,CountVectorizer,IDF,StringIndexer)

In [42]:
tokenizer=Tokenizer(inputCol='text',outputCol="tok_text")
stop_remove=StopWordsRemover(inputCol="tok_text",outputCol="stop_tok")
countVec=CountVectorizer(inputCol="stop_tok",outputCol="c_vec")
idf=IDF(inputCol="c_vec",outputCol="tf_idf")
ham_spam_to_numeric=StringIndexer(inputCol="class",outputCol="Label")

In [43]:
from pyspark.ml.feature import VectorAssembler
clean_up=VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')


In [46]:
from pyspark.ml.classification import NaiveBayes
nb=NaiveBayes()

In [48]:
from pyspark.ml import Pipeline
data_pipe=Pipeline(stages=[ham_spam_to_numeric,tokenizer,stop_remove,countVec,idf,clean_up])

In [50]:
cleaner=data_pipe.fit(data)
clean_data=cleaner.transform(data)
clean_data.show()

+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|class|                text|length|Label|          token_text|          stop_token|               c_vec|              tf_idf|            features|
+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|  ham|Go until jurong p...|   111|  0.0|[go, until, juron...|[go, jurong, poin...|(13423,[7,11,31,6...|(13423,[7,11,31,6...|(13424,[7,11,31,6...|
|  ham|Ok lar... Joking ...|    29|  0.0|[ok, lar..., joki...|[ok, lar..., joki...|(13423,[0,24,297,...|(13423,[0,24,297,...|(13424,[0,24,297,...|
| spam|Free entry in 2 a...|   155|  1.0|[free, entry, in,...|[free, entry, 2, ...|(13423,[2,13,19,3...|(13423,[2,13,19,3...|(13424,[2,13,19,3...|
|  ham|U dun say so earl...|    49|  0.0|[u, dun, say, so,...|[u, dun, say, ear...|(13423,[0,70,80,1...|(13423,[0,70,8

In [51]:
clean_data=clean_data.select("label","features")
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [53]:
train,test=clean_data.randomSplit([0.7,0.3])
smsspamdetection=nb.fit(train)

In [55]:
test_results=smsspamdetection.transform(test)
test_results.show(100)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,3,9,1...|[-571.20407774666...|[1.0,7.9750461227...|       0.0|
|  0.0|(13424,[0,1,5,20,...|[-803.82806067292...|[1.0,1.6864153818...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-1191.8108162686...|[1.0,5.1393290298...|       0.0|
|  0.0|(13424,[0,1,7,15,...|[-666.41257273280...|[1.0,5.8277112376...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-538.48125090426...|[1.0,5.8038864386...|       0.0|
|  0.0|(13424,[0,1,11,32...|[-906.09428113212...|[1.0,1.3517888219...|       0.0|
|  0.0|(13424,[0,1,12,33...|[-446.00532169568...|[1.0,3.0647539072...|       0.0|
|  0.0|(13424,[0,1,14,18...|[-1380.0650595395...|[1.0,2.8343309525...|       0.0|
|  0.0|(13424,[0,1,20,27...|[-968.40547423133...|[0.99999999999999...|       0.0|
|  0.0|(13424,[0

In [56]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_eval=MulticlassClassificationEvaluator()
acc=acc_eval.evaluate(test_results)
print(acc)

0.9262641491794753
